In [1]:
from IPython.display import display, Math, Latex, HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''') 

## 2.Summary

In [2]:
#Load libraries:
from data_exploration import *

In [3]:
all_data=False
#all_data=True

In [4]:
#Set up test time interval:
time_interval='4w' #2w

#Set up starting point, by default if will start from current time
starting_point=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#starting point="2019-01-10 14:00:00"  # to set upl alternative starting point

In [5]:
if not all_data:#Set up test time interval:
    print("Time interval: ", time_interval)
    print("Starting point:",starting_point )
else:
    print("Selecting all data from database")

Time interval:  4w
Starting point: 2019-01-23 10:58:20


In [6]:
#Set up influxdb connection:
client, client_df = connect_to_influxdb()

### Download/Upload speed

In [7]:
if not all_data:
    query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf' AND time >= '"+starting_point+"'-"+time_interval+" AND DOWNLOAD>0;"
else:
    query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf' AND DOWNLOAD>0;"
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,table_name='SPEEDTEST_DOWNLOAD')
#download_df.head()

In [8]:
device_numbers_d=download_df['SK_PI'].unique()
device_numbers_d=list(map(int, device_numbers_d))
device_numbers_d= sorted(device_numbers_d)
#device_numbers

In [9]:
download_summary=mean_max_median_min_by1(download_df,'DOWNLOAD')
if not all_data:
    t="Download speed over the last "+time_interval+ " starting from "+ starting_point
else:
    t="Download speed"
download_line=go.Scatter(x=device_numbers_d,y=[50] * len(device_numbers_d), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_4traces(xvalues=download_summary["SK_PI"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         yvalues4=download_summary["min"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         name4="Min",
                         title=t,
                         ytitle="Mbps",
                         line=download_line,
                         stack=False)

In [10]:
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title=t, 
               ytitle="Mbps",downloadline=True)

In [11]:
download_df["below50"]=0
download_df.loc[(download_df["DOWNLOAD"]<50),"below50"]=1
#download_df.head()

In [12]:
summary_download=download_df.groupby("SK_PI")['below50'].count().reset_index()
summary_download.rename(columns={'below50':'download_count'}, inplace=True)
subset_below50=download_df[download_df['below50']==1]
summary_below50=subset_below50.groupby("SK_PI")['below50'].count().reset_index()
summary_below50.rename(columns={'below50':'download_below50'}, inplace=True)
summary_download=pd.merge(summary_download, summary_below50,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_download.fillna(0, inplace=True)
summary_download['download_below50_percent']=round(summary_download['download_below50']/summary_download['download_count']*100)
summary_download=summary_download.sort_values(by=['SK_PI'], ascending=[True])
summary_download


,SK_PI,download_count,download_below50,download_below50_percent
0,3,171,0.0,0.0
1,5,172,172.0,100.0
2,7,172,172.0,100.0
3,8,24,0.0,0.0
4,9,172,171.0,99.0
5,10,155,155.0,100.0
6,11,172,171.0,99.0
7,12,172,172.0,100.0
8,14,171,53.0,31.0
9,15,172,172.0,100.0


In [13]:
device_number=14
subset=download_df[download_df["SK_PI"]==device_number]
hist_data = [subset['DOWNLOAD']]
group_labels = ['device '+str(device_number)+ ' download speed']
import plotly.figure_factory as ff
fig = ff.create_distplot(hist_data, group_labels,bin_size=10)
iplot(fig)

![](images/grafana_download14.png)

In [14]:
download_df["hour"]=pd.to_numeric(download_df["time"].dt.hour)

In [15]:
subset=download_df[download_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='DOWNLOAD',sort_value='hour',
               title="Download speed by hour for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",downloadline=True)

In [16]:
download_df["weekday"]=download_df["time"].dt.weekday_name
download_df["weekday"] = pd.Categorical(download_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [17]:
subset=download_df[download_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='DOWNLOAD',sort_value='weekday',
               title="Download speed by day of the week for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",downloadline=True, weekdays=True, jitter=True)

In [18]:
#devices_download_below50=summary_download[summary_download['download_below50_percent']>50]['SK_PI'].unique()
#hist_data=[]
#group_labels=[]
#colors_d=[]
#for device_number in devices_download_below50:
#    hist_data.append(download_df[download_df["SK_PI"]==device_number]["DOWNLOAD"])
#    group_labels.append(device_number)
#    colors_d.append(colors[device_number])
#
#fig = ff.create_distplot(hist_data, group_labels, colors=colors_d,
#                         bin_size=1, show_rug=False)
#
#fig['layout'].update(title='Download speed: distribution for all devices')
#iplot(fig)

In [19]:
if not all_data:
    query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND time >= '"+starting_point+"'-"+time_interval+" AND UPLOAD>0;"
else:
    query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_UPLOAD')
#upload_df.head()

In [20]:
device_numbers_u=upload_df['SK_PI'].unique()
device_numbers_u=list(map(int, device_numbers_u))
device_numbers_u= sorted(device_numbers_u)
#device_numbers_u

In [21]:
upload_summary=mean_max_median_min_by1(upload_df,'UPLOAD')
if not all_data:
    t="Upload speed over the last "+time_interval+ " starting from "+ starting_point
else:
    t="Upload speed"
upload_line=go.Scatter(x=device_numbers_d,y=[10] * len(device_numbers_u), mode='markers',marker=dict(color='red'), name='10Mps')
combined_bar_plot_4traces(xvalues=upload_summary["SK_PI"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         yvalues4=upload_summary["min"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         name4="Min",
                         title=t,
                         ytitle="Mbps",
                         line=upload_line,
                         stack=False)

In [22]:
simple_boxplot(dataframe=upload_df,plot_value='UPLOAD',sort_value='SK_PI',
               title=t, 
               ytitle="Mbps",uploadline=True)

In [23]:
upload_df["below10"]=0
upload_df.loc[(upload_df["UPLOAD"]<10),"below10"]=1
#upload_df.head()

In [24]:
summary_upload=upload_df.groupby("SK_PI")['below10'].count().reset_index()
summary_upload.rename(columns={'below10':'upload_count'}, inplace=True)
subset_below10=upload_df[upload_df['below10']==1]
summary_below10=subset_below10.groupby("SK_PI")['below10'].count().reset_index()
summary_below10.rename(columns={'below10':'upload_below10'}, inplace=True)
summary_upload=pd.merge(summary_upload, summary_below10,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_upload.fillna(0, inplace=True)
summary_upload['upload_below10_percent']=round(summary_upload['upload_below10']/summary_upload['upload_count']*100)
summary_upload=summary_upload.sort_values(by=['SK_PI'], ascending=[ True])
summary_upload

,SK_PI,upload_count,upload_below10,upload_below10_percent
0,3,171,0.0,0.0
1,5,172,172.0,100.0
2,7,172,130.0,76.0
3,8,24,0.0,0.0
4,9,172,0.0,0.0
5,10,155,155.0,100.0
6,11,172,0.0,0.0
7,12,172,170.0,99.0
8,14,171,6.0,4.0
9,15,172,0.0,0.0


In [25]:
device_number=7
subset=upload_df[upload_df["SK_PI"]==device_number]
hist_data = [subset['UPLOAD']]
group_labels = ['device '+str(device_number)+ ' upload speed']
import plotly.figure_factory as ff
fig = ff.create_distplot(hist_data, group_labels,bin_size=0.25)
iplot(fig)


![](images/grafana_upload7.png)

In [26]:
upload_df["hour"]=pd.to_numeric(upload_df["time"].dt.hour)
subset=upload_df[upload_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='hour',
               title="Upload speed by hour for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",uploadline=True)

In [27]:
upload_df["weekday"]=upload_df["time"].dt.weekday_name
upload_df["weekday"] = pd.Categorical(upload_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [28]:
subset=upload_df[upload_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='weekday',
               title="Upload speed by day of the week for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",uploadline=True, weekdays=True, jitter=True)

In [29]:
#devices_upload_below10=summary_upload[summary_upload['upload_below10_percent']>50]['SK_PI'].unique()
#hist_data=[]
#group_labels=[]
#colors_d=[]
#for device_number in devices_upload_below10:
#    hist_data.append(upload_df[upload_df["SK_PI"]==device_number]["UPLOAD"])
#    group_labels.append(device_number)
#    colors_d.append(colors[device_number])
#
#fig = ff.create_distplot(hist_data, group_labels, colors=colors_d,
#                         bin_size=0.25, show_rug=False)
#
# Add title
#fig['layout'].update(title='Upload speed: distribution for all devices')
#iplot(fig)

In [30]:
summary_upload_download=pd.merge(summary_upload, summary_download,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_upload_download=summary_upload_download.sort_values(by=['download_below50_percent','upload_below10_percent','SK_PI'], ascending=[False, False, True])
summary_upload_download[['SK_PI','upload_below10_percent','download_below50_percent']]

,SK_PI,upload_below10_percent,download_below50_percent
1,5,100.0,100.0
5,10,100.0,100.0
10,16,100.0,100.0
7,12,99.0,100.0
12,18,97.0,100.0
2,7,76.0,100.0
9,15,0.0,100.0
11,17,0.0,100.0
4,9,0.0,99.0
6,11,0.0,99.0


In [31]:
d = {'SK_PI': [7,12,14,17,11,16,10,18], 'lat': [51.7834662, 50.366367,49.81892,56.5020674,49.13794,
                                                  50.533,50.9080366,56.0867787], 'long': [-96.6966330,-96.613671,
                                                -98.52097,-94.2086756,-97.24325,-100.3155,-98.5971688,-96.0936590],
                                                'name':['Bloodvein','Brokenhead','Dakota Plains','Fox Lake','Ginew',
                                                        'Keeseekoowenin','Lake Manitoba','York Landing']}
coordinates_df = pd.DataFrame(data=d)
summary_ud_loc=pd.merge(summary_upload_download, coordinates_df,  how='right', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_ud_loc

,SK_PI,upload_count,upload_below10,upload_below10_percent,download_count,download_below50,download_below50_percent,lat,long,name
0,10,155,155.0,100.0,155,155.0,100.0,50.908037,-98.597169,Lake Manitoba
1,16,161,161.0,100.0,161,161.0,100.0,50.533000,-100.315500,Keeseekoowenin
2,12,172,170.0,99.0,172,172.0,100.0,50.366367,-96.613671,Brokenhead
3,18,168,163.0,97.0,168,168.0,100.0,56.086779,-96.093659,York Landing
4,7,172,130.0,76.0,172,172.0,100.0,51.783466,-96.696633,Bloodvein
5,17,170,0.0,0.0,170,170.0,100.0,56.502067,-94.208676,Fox Lake
6,11,172,0.0,0.0,172,171.0,99.0,49.137940,-97.243250,Ginew
7,14,171,6.0,4.0,171,53.0,31.0,49.818920,-98.520970,Dakota Plains


In [32]:
scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

summary_ud_loc['text'] = summary_ud_loc['SK_PI'].astype(str)+'.'+summary_ud_loc['name'] +': Download speed '+\
                         summary_ud_loc['download_below50_percent'].astype(str)+\
                        '% below 50Mbs, Upload speed ' + summary_ud_loc['upload_below10_percent'].astype(str) + '% below 10Mbs'
data = [ dict(
        type = 'scattergeo',
        locationmode = 'north america',
        lon = summary_ud_loc['long'],
        lat = summary_ud_loc['lat'],
        text = summary_ud_loc['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale='Greens',
            #colorscale = scl,
            cmin = 0,
            color = summary_ud_loc['download_below50_percent'],
            cmax = summary_ud_loc['download_below50_percent'].max(),
            colorbar=dict(
               # title="Percentage of download speed data below 50Mbs"
            )
        ))]

layout = dict(
        title = 'Devices colored by download speed <50Mbs percentage',
        colorbar = True,
        geo = dict(
            scope = 'north america',
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 50,
            projection = dict(
                type = 'kavrayskiy7',
            ),
             lonaxis = dict(
                gridwidth = 2,
                range= [ -120, -70 ],
                dtick = 10
            ),
            lataxis = dict (
                range= [ 42.0, 67.0 ],
                dtick = 10
            )
        ),
    )

fig = dict( data=data, layout=layout )
iplot( fig, validate=False)

### t-test for ping latency?